# Setup

In [1]:
import numpy as np
import pandas as pd
import dataclasses
import enum
import typing
import random
import fuzzy_hash_lib
import feature_encoding.base
import feature_encoding.threshold
import feature_encoding.direct
import feature_encoding.gray

from eeg_auth_models_framework import data, pre_process, features, processor, normalization
from eeg_auth_models_framework.utils import conversion

# Constants

In [2]:
AUTHENTICATION_THRESHOLDS = [10, 20, 30, 40, 50, 60, 70, 80, 90]
DATASET_SAMPLE_FREQ_HZ = 200
DATA_CHANNEL_NAMES = ['T7','F8','Cz','P4']
FREQUENCIES = [
    pre_process.FrequencyBand(lower=8.0, upper=12.0, label='Alpha'),
    pre_process.FrequencyBand(lower=12.0, upper=35.0, label='Beta'),
    pre_process.FrequencyBand(lower=4.0, upper=8.0, label='Theta'),
    pre_process.FrequencyBand(lower=35.0, upper=None, label='Gamma'),
    pre_process.FrequencyBand(lower=None, upper=None, label='Raw'),
]
WINDOW_SIZE = 1200
WINDOW_OVERLAP = 0
BINARY_THRESHOLD = 127
RESCALE_LOWER = 0
RESCALE_UPPER = 255
RANDOM_SEED = 100000000000
RANDOM_GENERATOR = random.Random(RANDOM_SEED)

# Utilities

In [3]:
class TestResultType(enum.Enum):
    TRUE_POSITIVE = enum.auto()
    FALSE_POSITIVE = enum.auto()
    FALSE_NEGATIVE = enum.auto()
    TRUE_NEGATIVE = enum.auto()


@dataclasses.dataclass
class TestResultsSummary:
    true_positives: int = 0
    false_positives: int = 0
    false_negatives: int = 0
    true_negatives: int = 0
    
    @classmethod
    def merge_summaries(cls, 
                        summary_a: 'TestResultsSummary', 
                        summary_b: 'TestResultsSummary') -> 'TestResultsSummary':
        return TestResultsSummary(
            true_positives=summary_a.true_positives + summary_b.true_positives,
            false_positives=summary_a.false_positives + summary_b.false_positives,
            false_negatives=summary_a.false_negatives + summary_b.false_negatives,
            true_negatives=summary_a.true_negatives + summary_b.true_negatives
        )
    
    def increment_count(self, result_type: TestResultType):
        if result_type == TestResultType.TRUE_POSITIVE:
            self.true_positives += 1
        elif result_type == TestResultType.FALSE_POSITIVE:
            self.false_positives += 1
        elif result_type == TestResultType.FALSE_NEGATIVE:
            self.false_negatives += 1
        else:
            self.true_negatives += 1
    
    @property
    def accuracy(self) -> float:
        hits = self.true_positives + self.true_negatives
        total = (
            self.true_positives + self.true_negatives + 
            self.false_positives + self.false_negatives
        )
        return hits / total

    @property
    def false_accept_rate(self) -> float:
        return self.false_positives / (self.false_positives + self.true_negatives)
    
    @property
    def false_reject_rate(self):
        return self.false_negatives / (self.false_negatives + self.true_positives)


@dataclasses.dataclass
class HashTest:
    expected_result: bool
    threshold: int
    if_expected: TestResultType
    if_unexpected: TestResultType
    hashes: typing.Tuple[fuzzy_hash_lib.FuzzyHash, fuzzy_hash_lib.FuzzyHash]
    
    def run_test(self):
        similarity = fuzzy_hash_lib.FuzzyHash.compare(self.hashes[0], self.hashes[1])
        is_match = similarity >= self.threshold
        if is_match != self.expected_result:
            return self.if_unexpected
        return self.if_expected
    
@dataclasses.dataclass
class ThresholdTestSet:
    threshold: int
    template_hash: fuzzy_hash_lib.FuzzyHash
    positive_cases: typing.List[fuzzy_hash_lib.FuzzyHash]
    negative_cases: typing.List[fuzzy_hash_lib.FuzzyHash]
    
@dataclasses.dataclass
class SubjectTestSet:
    subject_id: str
    threshold_tests: typing.List[ThresholdTestSet]

# Configuration

In [4]:
downloader = data.AuditoryDataDownloader()
reader = data.AuditoryDataReader()
converter = conversion.MNEDataFrameConverter(
    channels=DATA_CHANNEL_NAMES, 
    sample_frequency=DATASET_SAMPLE_FREQ_HZ
)

# Data Processing Setup

## Template Hash Processor

### Pre-Processing Steps

In [5]:
template_pre_process_steps = pre_process.PreProcessingPipeline([
    pre_process.EEGBandpassFilterStep(
        FREQUENCIES,
        converter
    )
])

### Feature Extraction Steps

In [6]:
template_feature_extraction_steps = features.FeatureExtractPipeline([
    features.StatisticalFeatureExtractor([
        features.StatisticalFeature.MIN,
        features.StatisticalFeature.MAX,
        features.StatisticalFeature.MEAN,
        features.StatisticalFeature.ZERO_CROSSING_RATE
    ])
])

### Normalization Steps

In [7]:
template_normalization_steps = normalization.NormalizationPipeline([
    normalization.RescaleNormalizationStep(RESCALE_LOWER, RESCALE_UPPER),
    normalization.HistogramEqualizationStep(RESCALE_LOWER, RESCALE_UPPER)
])

### Data Processor Setup

In [8]:
template_data_processor = processor.DataProcessor(
    pre_process=template_pre_process_steps,
    feature_extraction=template_feature_extraction_steps,
    normalization=template_normalization_steps
)

## Sample Data Processor

### Pre-Processing Steps

In [9]:
pre_process_steps = pre_process.PreProcessingPipeline([
    pre_process.EEGBandpassFilterStep(
        FREQUENCIES,
        converter
    ),
    pre_process.DataWindowStep(WINDOW_SIZE, WINDOW_OVERLAP)
])

### Feature Extraction Steps

In [10]:
feature_extraction_steps = features.FeatureExtractPipeline([
    features.StatisticalFeatureExtractor([
        features.StatisticalFeature.MIN,
        features.StatisticalFeature.MAX,
        features.StatisticalFeature.MEAN,
        features.StatisticalFeature.ZERO_CROSSING_RATE
    ])
])

### Normalization Steps

In [11]:
normalization_steps = normalization.NormalizationPipeline([
    normalization.RescaleNormalizationStep(RESCALE_LOWER, RESCALE_UPPER),
    normalization.HistogramEqualizationStep(RESCALE_LOWER, RESCALE_UPPER)
])

### Data Processor Setup

In [12]:
data_processor = processor.DataProcessor(
    pre_process=pre_process_steps,
    feature_extraction=feature_extraction_steps,
    normalization=normalization_steps
)

# Subject Data

In [13]:
data_path = downloader.retrieve()
subject_data_map = reader.format_data(data_path)

# Processing

## Metadata

In [14]:
metadata_map = {subject: data_processor.extract_metadata(subject_data_map[subject]) for subject in subject_data_map}

Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=40114
    Range : 0 ... 40113 =      0.000 ...   200.565 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ..

## Template Hash Setup

### Processing

In [15]:
template_data_map = {
    subject: template_data_processor.process(subject_data_map[subject], metadata_map[subject]) 
    for subject in subject_data_map
}

Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=40114
    Range : 0 ... 40113 =      0.000 ...   200.565 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ..

### Hashing

In [16]:
SubjectTemplatePair = typing.Tuple[str, fuzzy_hash_lib.FuzzyHash]
SubjectProcessedData = typing.Dict[str, typing.List[np.ndarray]]

In [17]:
def hash_vectors(vectors_to_hash: typing.List[np.ndarray], encoder: feature_encoding.base.BinaryEncoder) -> typing.List[fuzzy_hash_lib.FuzzyHash]:
    binary_vectors = [encoder.encode(v) for v in vectors_to_hash]
    return [fuzzy_hash_lib.FuzzyHash.from_text(bv) for bv in binary_vectors]


def iter_template_hashes(template_data: SubjectProcessedData, encoder: feature_encoding.base.BinaryEncoder) -> typing.Iterator[SubjectTemplatePair]:
    for subject in template_data:
        hashes_list = hash_vectors(template_data[subject], encoder)
        if len(hashes_list) != 1:
            print(
                f'[warning] There should be only one template hash per subject, '
                f'but subject {subject} has {len(hashes_list)} template hashes'
            )
            continue
        yield subject, hashes_list[0]

## Sample Hashes Setup

### Subject Data

In [18]:
processed_data_map = {
    subject: data_processor.process(subject_data_map[subject], metadata_map[subject])
    for subject in subject_data_map
}

Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=40114
    Range : 0 ... 40113 =      0.000 ...   200.565 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ..

### Hashing

In [19]:
SubjectSamplesPair = typing.Tuple[str, typing.List[fuzzy_hash_lib.FuzzyHash]]

In [20]:
def iter_sample_hashes(data_map: SubjectProcessedData, encoder: feature_encoding.base.BinaryEncoder) -> typing.Iterator[SubjectSamplesPair]:
    for subject in data_map:
        vectors = hash_vectors(data_map[subject], encoder)
        yield subject, vectors

# Test Set Assembly

## Gathering Test Sets

In [21]:
def iter_subject_test_sets(template_data: SubjectProcessedData, sample_data: SubjectProcessedData, encoder: feature_encoding.base.BinaryEncoder) -> typing.Iterator[SubjectTestSet]:
    for target_subject, template_hash in iter_template_hashes(template_data, encoder):
        threshold_tests = []
        for threshold in AUTHENTICATION_THRESHOLDS:
            threshold_test_set = ThresholdTestSet(
                positive_cases=[], negative_cases=[], 
                template_hash=template_hash, threshold=threshold
            )
            for subject, hashes in iter_sample_hashes(sample_data, encoder):
                if subject == target_subject:
                    threshold_test_set.positive_cases.extend(hashes)
                else:
                    threshold_test_set.negative_cases.extend(hashes)
            threshold_tests.append(threshold_test_set)
        yield SubjectTestSet(
            subject_id=target_subject,
            threshold_tests=threshold_tests
        )

## Generating Hash Tests

In [22]:
ThresholdTestsPair = typing.Tuple[str, typing.List[HashTest]]

In [23]:
def make_hash_tests(test_set: ThresholdTestSet) -> typing.List[HashTest]:
    tests = []
    sample_size = min(len(test_set.positive_cases), len(test_set.negative_cases))
    should_match_cases: typing.List[fuzzy_hash_lib.FuzzyHash] = RANDOM_GENERATOR.sample(test_set.positive_cases, sample_size)
    should_not_match_cases: typing.List[fuzzy_hash_lib.FuzzyHash] = RANDOM_GENERATOR.sample(test_set.negative_cases, sample_size)
    for sample in should_match_cases:
        tests.append(
            HashTest(
                expected_result=True, 
                threshold=test_set.threshold, 
                hashes=(test_set.template_hash, sample),
                if_expected=TestResultType.TRUE_POSITIVE,
                if_unexpected=TestResultType.FALSE_NEGATIVE
            )
        )
    for sample in should_not_match_cases:
        tests.append(
            HashTest(
                expected_result=False, 
                threshold=test_set.threshold, 
                hashes=(test_set.template_hash, sample),
                if_expected=TestResultType.TRUE_NEGATIVE,
                if_unexpected=TestResultType.FALSE_POSITIVE
            )
        )
    return tests


def iter_hash_tests(template_data: SubjectProcessedData, sample_data: SubjectProcessedData, encoder: feature_encoding.base.BinaryEncoder) -> ThresholdTestsPair:
    for subject_test_set in iter_subject_test_sets(template_data, sample_data, encoder):
        for threshold_test_data in subject_test_set.threshold_tests:
            yield threshold_test_data.threshold, make_hash_tests(threshold_test_data)

# Execute Tests

In [24]:
def run_hash_tests(hash_tests: typing.List[HashTest]) -> TestResultsSummary:
    summary = TestResultsSummary()
    for test in hash_tests:
        result_type = test.run_test()
        summary.increment_count(result_type)
    return summary

## Threshold-Based Encoding

In [25]:
threshold_results_map = {}
threshold_encoder = feature_encoding.threshold.ThresholdBinaryEncoder(BINARY_THRESHOLD)
threshold_test_args = (
    template_data_map, processed_data_map, threshold_encoder
)
for threshold_type, hash_test_data in iter_hash_tests(*threshold_test_args):
    if threshold_type not in threshold_results_map:
        threshold_results_map[threshold_type] = TestResultsSummary()
    threshold_results_map[threshold_type] = TestResultsSummary.merge_summaries(
        threshold_results_map[threshold_type],
        run_hash_tests(hash_test_data)
    )

In [26]:
threshold_results = []
for threshold_type, result_summary in threshold_results_map.items():
    threshold_results.append([
        threshold_type, result_summary.false_accept_rate, 
        result_summary.false_reject_rate, result_summary.accuracy
    ])
threshold_df = pd.DataFrame(threshold_results, columns=['Threshold', 'FAR', 'FRR', 'Accuracy'])
threshold_df

,Threshold,FAR,FRR,Accuracy
0,10,0.055690,0.181598,0.881356
1,20,0.055690,0.181598,0.881356
2,30,0.004843,0.283293,0.855932
3,40,0.004843,0.285714,0.854722
4,50,0.002421,0.285714,0.855932
5,60,0.004843,0.285714,0.854722
6,70,0.000000,0.285714,0.857143
7,80,0.002421,0.285714,0.855932
8,90,0.004843,0.285714,0.854722


## Direct Encoding

In [27]:
direct_results_map = {}
direct_encoder = feature_encoding.direct.DirectBinaryEncoder()
direct_test_args = (
    template_data_map, processed_data_map, direct_encoder
)
for threshold_type, hash_test_data in iter_hash_tests(*direct_test_args):
    if threshold_type not in direct_results_map:
        direct_results_map[threshold_type] = TestResultsSummary()
    direct_results_map[threshold_type] = TestResultsSummary.merge_summaries(
        direct_results_map[threshold_type],
        run_hash_tests(hash_test_data)
    )

In [28]:
direct_results = []
for threshold_type, result_summary in direct_results_map.items():
    direct_results.append([
        threshold_type, result_summary.false_accept_rate, 
        result_summary.false_reject_rate, result_summary.accuracy
    ])
direct_df = pd.DataFrame(direct_results, columns=['Threshold', 'FAR', 'FRR', 'Accuracy'])
direct_df

,Threshold,FAR,FRR,Accuracy
0,10,0.0,1.0,0.5
1,20,0.0,1.0,0.5
2,30,0.0,1.0,0.5
3,40,0.0,1.0,0.5
4,50,0.0,1.0,0.5
5,60,0.0,1.0,0.5
6,70,0.0,1.0,0.5
7,80,0.0,1.0,0.5
8,90,0.0,1.0,0.5


## Gray Encoding

In [29]:
gray_results_map = {}
gray_encoder = feature_encoding.gray.GrayCodeBinaryEncoder(8, round_to_whole=True)
gray_test_args = (
    template_data_map, processed_data_map, gray_encoder
)
for threshold_type, hash_test_data in iter_hash_tests(*gray_test_args):
    if threshold_type not in gray_results_map:
        gray_results_map[threshold_type] = TestResultsSummary()
    gray_results_map[threshold_type] = TestResultsSummary.merge_summaries(
        gray_results_map[threshold_type],
        run_hash_tests(hash_test_data)
    )

In [30]:
gray_results = []
for threshold_type, result_summary in gray_results_map.items():
    gray_results.append([
        threshold_type, result_summary.false_accept_rate, 
        result_summary.false_reject_rate, result_summary.accuracy
    ])
gray_df = pd.DataFrame(gray_results, columns=['Threshold', 'FAR', 'FRR', 'Accuracy'])
gray_df

,Threshold,FAR,FRR,Accuracy
0,10,0.0,1.0,0.5
1,20,0.0,1.0,0.5
2,30,0.0,1.0,0.5
3,40,0.0,1.0,0.5
4,50,0.0,1.0,0.5
5,60,0.0,1.0,0.5
6,70,0.0,1.0,0.5
7,80,0.0,1.0,0.5
8,90,0.0,1.0,0.5
